In [1]:
import pandas as pd
import gmaps
import sys
import requests
import json
from config import gkey
import sqlalchemy
import numpy as np
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.ext.declarative import declarative_base

### Extract CSVs into DataFrames

In [2]:
confirmados_file = "Confirmados.csv"
defunciones_file = "Defunciones.csv"
negativos_file = "Negativos.csv"

### Transform DataFrames

In [36]:
confirmados_data = pd.read_csv(confirmados_file)
confirmados_df = pd.DataFrame(confirmados_data)
confirmados_df = confirmados_df.dropna(how='all')
confirmados_df.insert(3, "status", "confirmed", True) 
confirmados_df.rename(columns = {'nombre':'estado'}, inplace = True)
c1_df = confirmados_df.drop([0])
#The next line is to add consecutive numbers
c1_df['row_number'] = np.arange(len(c1_df))
c1_df.head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_17,April_18,April_19,April_20,April_21,April_22,Unnamed: 97,Unnamed: 98,Unnamed: 99,row_number
1,1.0,AGUASCALIENTES,Mexico,confirmed,NaN,NaN,0.0,0.0,0.0,0.0,...,9.0,6.0,1.0,1.0,0.0,2.0,NaN,NaN,NaN,0
2,2.0,BAJA CALIFORNIA,Mexico,confirmed,NaN,NaN,0.0,0.0,0.0,0.0,...,34.0,15.0,6.0,23.0,1.0,0.0,NaN,NaN,NaN,1
3,3.0,BAJA CALIFORNIA SUR,Mexico,confirmed,NaN,NaN,0.0,0.0,0.0,0.0,...,5.0,1.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,2
4,4.0,CAMPECHE,Mexico,confirmed,NaN,NaN,0.0,0.0,0.0,0.0,...,1.0,5.0,1.0,1.0,2.0,0.0,NaN,NaN,NaN,3
5,5.0,COAHUILA,Mexico,confirmed,NaN,NaN,0.0,0.0,0.0,0.0,...,8.0,1.0,2.0,2.0,12.0,0.0,NaN,NaN,NaN,4


In [27]:
#Nacional confirmados
na1 = confirmados_df.loc[confirmados_df['estado'] == "Nacional"]
na1

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_16,April_17,April_18,April_19,April_20,April_21,April_22,Unnamed: 97,Unnamed: 98,Unnamed: 99
0,0.0,Nacional,Mexico,confirmed,NaN,NaN,0.0,0.0,0.0,0.0,...,654.0,663.0,419.0,343.0,569.0,265.0,67.0,NaN,NaN,NaN


In [23]:
negativos_data = pd.read_csv(negativos_file)
negativos_df = pd.DataFrame(negativos_data)
negativos_df = negativos_df.dropna(how='all')
negativos_df.insert(3, "status","negativos", True)
n1_df = negativos_df.drop([0])
#The next line is to add consecutive numbers
n1_df['row_number'] = np.arange(len(n1_df))
n1_df.head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
1,1,AGUASCALIENTES,Mexico,negativos,NaN,NaN,0,1,1,0,...,40,58,32,20,10,18,23,1,0,0
2,2,BAJA CALIFORNIA,Mexico,negativos,NaN,NaN,2,1,2,0,...,50,28,33,20,8,5,14,0,0,1
3,3,BAJA CALIFORNIA SUR,Mexico,negativos,NaN,NaN,0,0,0,0,...,33,36,23,10,3,3,8,9,3,2
4,4,CAMPECHE,Mexico,negativos,NaN,NaN,0,0,0,0,...,10,7,9,3,5,1,20,2,0,3
5,5,COAHUILA,Mexico,negativos,NaN,NaN,5,3,2,0,...,102,73,139,98,34,8,44,22,1,4


In [6]:
na2 = negativos_df.loc[negativos_df['estado'] == "Nacional"]
na2

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_13,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22
0,0,Nacional,Mexico,negativos,NaN,NaN,56,46,46,15,...,1822,1759,1505,1438,1565,633,520,1261,609,150


In [28]:
defunciones_data = pd.read_csv(defunciones_file)
defunciones_df = pd.DataFrame(defunciones_data)
defunciones_df = defunciones_df.dropna(how='all')
defunciones_df.insert(3, "status","defunciones", True)
d1_df = defunciones_df.drop([0])
#The next line is to add consecutive numbers
d1_df['row_number'] = np.arange(len(d1_df))
d1_df.head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
1,1,AGUASCALIENTES,Mexico,defunciones,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,BAJA CALIFORNIA,Mexico,defunciones,NaN,NaN,0,0,0,0,...,8,14,6,8,2,9,5,3,0,1
3,3,BAJA CALIFORNIA SUR,Mexico,defunciones,NaN,NaN,0,0,0,0,...,0,0,1,1,0,0,1,0,1,2
4,4,CAMPECHE,Mexico,defunciones,NaN,NaN,0,0,0,0,...,1,0,0,1,0,1,0,0,0,3
5,5,COAHUILA,Mexico,defunciones,NaN,NaN,0,0,0,0,...,3,1,3,0,0,1,3,1,1,4


In [8]:
na3 = defunciones_df.loc[defunciones_df['estado'] == "Nacional"]
na3

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_13,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22
0,0,Nacional,Mexico,defunciones,NaN,NaN,0,0,0,0,...,55,64,65,49,56,52,66,58,37,29


### Merge all DataFrames

In [29]:
#Merge by state
estados_df = pd.concat([c1_df,n1_df,d1_df], ignore_index=True)
estados_df.fillna('', inplace=True)


C:\Users\Adriana Lorena\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [37]:
#Google API for Lat and Lng
params = {"key": gkey}
for index, row in estados_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    state = row['estado']
    country = row['pais']
    params['address'] = f"{state},{country}"
    state_lat_lng = requests.get(base_url, params=params)
    state_lat_lng = state_lat_lng.json()
    estados_df.loc[index, "Lat"] = float(state_lat_lng["results"][0]["geometry"]["location"]["lat"])
    estados_df.loc[index, "Long"] = float(state_lat_lng["results"][0]["geometry"]["location"]["lng"])

#The next line is to add consecutive numbers
estados_df['row_number'] = np.arange(len(estados_df))
estados_df.head()

,April_1,April_10,April_11,April_12,April_13,April_14,April_15,April_16,April_17,April_18,...,March_8,March_9,Unnamed: 97,Unnamed: 98,Unnamed: 99,estado,fips,pais,row_number,status
0,0.0,2.0,1.0,2.0,6.0,0.0,9.0,4.0,9.0,6.0,...,0.0,0.0,,,,AGUASCALIENTES,1.0,Mexico,0,confirmed
1,61.0,45.0,38.0,37.0,78.0,102.0,69.0,58.0,34.0,15.0,...,0.0,0.0,,,,BAJA CALIFORNIA,2.0,Mexico,1,confirmed
2,5.0,9.0,7.0,5.0,11.0,14.0,5.0,9.0,5.0,1.0,...,0.0,0.0,,,,BAJA CALIFORNIA SUR,3.0,Mexico,2,confirmed
3,0.0,1.0,5.0,2.0,2.0,4.0,2.0,4.0,1.0,5.0,...,1.0,0.0,,,,CAMPECHE,4.0,Mexico,3,confirmed
4,18.0,11.0,7.0,5.0,14.0,14.0,13.0,4.0,8.0,1.0,...,0.0,0.0,,,,COAHUILA,5.0,Mexico,4,confirmed


In [38]:
#Merge by nacional
nacional_df = pd.concat([na1,na2,na3], ignore_index=True)
#The next line is to add consecutive numbers
nacional_df['row_number'] = np.arange(len(nacional_df))
nacional_df.head()

C:\Users\Adriana Lorena\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,April_1,April_10,April_11,April_12,April_13,April_14,April_15,April_16,April_17,April_18,...,March_8,March_9,Unnamed: 97,Unnamed: 98,Unnamed: 99,estado,fips,pais,status,row_number
0,295.0,426.0,359.0,368.0,702.0,702.0,686.0,654.0,663.0,419.0,...,4.0,1.0,NaN,NaN,NaN,Nacional,0.0,Mexico,confirmed,0
1,1209.0,1411.0,765.0,666.0,1822.0,1759.0,1505.0,1438.0,1565.0,633.0,...,31.0,77.0,NaN,NaN,NaN,Nacional,0.0,Mexico,negativos,1
2,18.0,51.0,54.0,52.0,55.0,64.0,65.0,49.0,56.0,52.0,...,0.0,0.0,NaN,NaN,NaN,Nacional,0.0,Mexico,defunciones,2


### Create database connection

In [30]:
rds_connection_string = "postgres:postgres@localhost:5432/Project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [41]:
# Confirm tables
engine.table_names()

['confirmed',
 'death',
 'recovered',
 'all_countries',
 'negativos',
 'defunciones',
 'confirmados',
 'estados',
 'nacional']

### Load DataFrames into database

In [32]:
n1_df.to_sql(name='negativos', con=engine, if_exists='append', index=False)

In [33]:
d1_df.to_sql(name='defunciones', con=engine, if_exists='append', index=False)

In [34]:
c1_df.to_sql(name='confirmados', con=engine, if_exists='append', index=False)

In [39]:
estados_df.to_sql(name='estados', con=engine, if_exists='append', index=False)

In [40]:
nacional_df.to_sql(name='nacional', con=engine, if_exists='append', index=False)

In [42]:
#Print the sql table with a query
pd.read_sql_query('select * from negativos', con=engine).head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
0,1,AGUASCALIENTES,Mexico,negativos,None,None,0,1,1,0,...,40,58,32,20,10,18,23,1,0,0
1,2,BAJA CALIFORNIA,Mexico,negativos,None,None,2,1,2,0,...,50,28,33,20,8,5,14,0,0,1
2,3,BAJA CALIFORNIA SUR,Mexico,negativos,None,None,0,0,0,0,...,33,36,23,10,3,3,8,9,3,2
3,4,CAMPECHE,Mexico,negativos,None,None,0,0,0,0,...,10,7,9,3,5,1,20,2,0,3
4,5,COAHUILA,Mexico,negativos,None,None,5,3,2,0,...,102,73,139,98,34,8,44,22,1,4


In [43]:
pd.read_sql_query('select * from defunciones', con=engine).head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
0,1,AGUASCALIENTES,Mexico,defunciones,None,None,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,BAJA CALIFORNIA,Mexico,defunciones,None,None,0,0,0,0,...,8,14,6,8,2,9,5,3,0,1
2,3,BAJA CALIFORNIA SUR,Mexico,defunciones,None,None,0,0,0,0,...,0,0,1,1,0,0,1,0,1,2
3,4,CAMPECHE,Mexico,defunciones,None,None,0,0,0,0,...,1,0,0,1,0,1,0,0,0,3
4,5,COAHUILA,Mexico,defunciones,None,None,0,0,0,0,...,3,1,3,0,0,1,3,1,1,4


In [44]:
pd.read_sql_query('select * from confirmados', con=engine).head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_17,April_18,April_19,April_20,April_21,April_22,Unnamed: 97,Unnamed: 98,Unnamed: 99,row_number
0,1.0,AGUASCALIENTES,Mexico,confirmed,None,None,0.0,0.0,0.0,0.0,...,9.0,6.0,1.0,1.0,0.0,2.0,None,None,None,0
1,2.0,BAJA CALIFORNIA,Mexico,confirmed,None,None,0.0,0.0,0.0,0.0,...,34.0,15.0,6.0,23.0,1.0,0.0,None,None,None,1
2,3.0,BAJA CALIFORNIA SUR,Mexico,confirmed,None,None,0.0,0.0,0.0,0.0,...,5.0,1.0,1.0,0.0,1.0,0.0,None,None,None,2
3,4.0,CAMPECHE,Mexico,confirmed,None,None,0.0,0.0,0.0,0.0,...,1.0,5.0,1.0,1.0,2.0,0.0,None,None,None,3
4,5.0,COAHUILA,Mexico,confirmed,None,None,0.0,0.0,0.0,0.0,...,8.0,1.0,2.0,2.0,12.0,0.0,None,None,None,4


In [45]:
pd.read_sql_query('select * from estados', con=engine).head()

,April_1,April_10,April_11,April_12,April_13,April_14,April_15,April_16,April_17,April_18,...,March_8,March_9,Unnamed: 97,Unnamed: 98,Unnamed: 99,estado,fips,pais,row_number,status
0,0.0,2.0,1.0,2.0,6.0,0.0,9.0,4.0,9.0,6.0,...,0.0,0.0,,,,AGUASCALIENTES,1.0,Mexico,0,confirmed
1,61.0,45.0,38.0,37.0,78.0,102.0,69.0,58.0,34.0,15.0,...,0.0,0.0,,,,BAJA CALIFORNIA,2.0,Mexico,1,confirmed
2,5.0,9.0,7.0,5.0,11.0,14.0,5.0,9.0,5.0,1.0,...,0.0,0.0,,,,BAJA CALIFORNIA SUR,3.0,Mexico,2,confirmed
3,0.0,1.0,5.0,2.0,2.0,4.0,2.0,4.0,1.0,5.0,...,1.0,0.0,,,,CAMPECHE,4.0,Mexico,3,confirmed
4,18.0,11.0,7.0,5.0,14.0,14.0,13.0,4.0,8.0,1.0,...,0.0,0.0,,,,COAHUILA,5.0,Mexico,4,confirmed


In [46]:
pd.read_sql_query('select * from nacional', con=engine).head()

,April_1,April_10,April_11,April_12,April_13,April_14,April_15,April_16,April_17,April_18,...,March_8,March_9,Unnamed: 97,Unnamed: 98,Unnamed: 99,estado,fips,pais,status,row_number
0,295.0,426.0,359.0,368.0,702.0,702.0,686.0,654.0,663.0,419.0,...,4.0,1.0,None,None,None,Nacional,0.0,Mexico,confirmed,0
1,1209.0,1411.0,765.0,666.0,1822.0,1759.0,1505.0,1438.0,1565.0,633.0,...,31.0,77.0,None,None,None,Nacional,0.0,Mexico,negativos,1
2,18.0,51.0,54.0,52.0,55.0,64.0,65.0,49.0,56.0,52.0,...,0.0,0.0,None,None,None,Nacional,0.0,Mexico,defunciones,2


In [47]:
#Set a primary key in an existing table
engine.execute('alter table negativos add primary key(row_number)')

In [48]:
engine.execute('alter table defunciones add primary key(row_number)')

In [49]:
engine.execute('alter table confirmados add primary key(row_number)')

In [50]:
engine.execute('alter table estados add primary key(row_number)')

In [51]:
engine.execute('alter table nacional add primary key(row_number)')

In [52]:
#THis is just to check if the tables are been mapped
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [53]:
Base.classes.keys()

['confirmed',
 'death',
 'recovered',
 'all_countries',
 'negativos',
 'defunciones',
 'confirmados',
 'estados',
 'nacional']

### SQL Tables

# Just change the table name 
create table confirmados/sospechosos/negativos/defunciones/estados/nacional
(
fips varchar(100)
estado varchar(100),
pais varchar(100),
status varchar(100),
Lat int,
Long int,
"January_22" string,
"January_23" string,
"January_24" string,
"January_25" string,
"January_26" string,
"January_27" string,
"January_28" string,
"January_29" string,
"January_30" string,
"January_31" string,
"Feburary_1" string,
"Feburary_2" string,
"Feburary_3" string,
"Feburary_4" string,
"Feburary_5" string,
"Feburary_6" string,
"Feburary_7" string,
"Feburary_8" string,
"Feburary_9" string,
"Feburary_10" string,
"Feburary_11" string,
"Feburary_12" string,
"Feburary_13" string,
"Feburary_14" string,
"Feburary_15" string,
"Feburary_16" string,
"Feburary_17" string,
"Feburary_18" string,
"Feburary_19" string,
"Feburary_20" string,
"Feburary_21" string,
"Feburary_22" string,
"Feburary_23" string,
"Feburary_24" string,
"Feburary_25" string,
"Feburary_26" string,
"Feburary_27" string,
"Feburary_28" string,
"Feburary_29" string,
"March_1" string,
"March_2" string,
"March_3" string,
"March_4" string,
"March_5" string,
"March_6" string,
"March_7" string,
"March_8" string,
"March_9" string,
"March_10" string,
"March_11" string,
"March_12" string,
"March_13" string,
"March_14" string,
"March_15" string,
"March_16" string,
"March_17" string,
"March_18" string,
"March_19" string,
"March_20" string,
"March_21" string,
"March_22" string,
"March_23" string,
"March_24" string,
"March_25" string,
"March_26" string,
"March_27" string,
"March_28" string,
"March_29" string,
"March_30" string,
"March_31" string,
"April_1" string,
"April_2" string,
"April_3" string,
"April_4" string,
"April_5" string,
"April_6" string,
"April_7" string,
"April_8" string,
"April_9" string,
"April_10" string,
"April_11" string,
"April_12" string,
"April_13" string,
"April_14" string,
"April_15" string,
"April_16" string,
"April_17" string,
"April_18" string,
"April_19" string,
"April_20" string,
"April_21" string,
"April_22" string
)